<a href="https://colab.research.google.com/github/AhmadAlabrash/NLP/blob/main/NLP_transform_dataset_to_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df = pd.read_csv("Fake_Real_Data.csv")

In [5]:
df['label_num'] = df.label.map({
    'Fake' : 0 ,
    'Real' : 1
})

In [2]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [6]:
nlp(df.Text[0]).vector

array([-0.6759837 ,  1.4263071 , -2.318466  , -0.451093  ,  2.8199325 ,
        0.85161996,  0.01045941,  3.6876464 , -1.1069572 , -0.89137274,
        4.7383623 ,  0.93590134, -2.4224715 ,  0.1756112 ,  1.1680135 ,
        1.358259  ,  0.968191  , -1.223655  , -0.95027757, -0.3604985 ,
        0.54369414, -0.48284245, -0.42516926, -1.8079175 , -0.63552403,
       -1.7014275 , -1.5113001 , -0.5503619 , -0.78364307,  0.83594626,
        0.8964651 , -0.1389701 , -0.7893373 , -0.42614973, -2.1502306 ,
       -0.92926306, -0.8751075 ,  0.8984946 ,  1.1605583 ,  1.0838758 ,
       -0.04290289,  0.0810288 , -0.04022507,  0.16692582, -1.4921017 ,
        1.1105057 ,  0.07551349, -3.346768  , -1.3400731 ,  2.8042486 ,
       -1.1094471 ,  0.2824121 ,  0.8654462 , -3.8102863 , -0.59461933,
        0.0378601 ,  0.2941968 ,  1.0687567 ,  0.99998826,  0.66275495,
        1.6435239 , -0.40380004, -0.6515654 , -1.4085007 ,  1.8625499 ,
        1.0395219 , -2.27062   , -2.883119  ,  0.42053694,  1.83

In [7]:
df['text_vectors']=df.Text.map(lambda x : nlp(x).vector)

In [12]:
df['text_vectors']

0       [-0.6759837, 1.4263071, -2.318466, -0.451093, ...
1       [-1.8355803, 1.3101058, -2.4919677, 1.0268308,...
2       [-1.9851209, 0.14389805, -2.4221718, 0.9133005...
3       [-2.7812982, -0.16120885, -1.609772, 1.3624227...
4       [-2.2010763, 0.9961637, -2.4088492, 1.128273, ...
                              ...                        
9895    [-1.6682401, 0.78006977, -2.2337353, -0.159771...
9896    [-1.9297235, 0.8007302, -1.8990824, 0.42668718...
9897    [-1.5289013, 1.0250993, -1.9861357, 0.4278564,...
9898    [-1.3928099, 0.7792715, -2.2072845, 0.13192406...
9899    [-1.5856593, -0.31769615, -0.9302766, 0.330447...
Name: text_vectors, Length: 9900, dtype: object

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.text_vectors,
    df.label_num,
    test_size=0.2
)

In [43]:
import numpy as np

x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [44]:
x_train

7982    [-1.1513766, 0.8472771, -1.6613288, 0.13114439...
2630    [-2.7477639, 1.2935811, -1.667479, 1.2355894, ...
6876    [-0.9504879, 1.0531021, -2.444918, -0.5767492,...
3941    [-1.8247339, 0.7739705, -1.8603653, 0.9636776,...
6718    [-0.99104637, 1.1280751, -1.6571653, -0.298668...
                              ...                        
5650    [-2.122114, 0.7669057, -2.4063985, 0.7626442, ...
4356    [-1.7900779, 0.8168002, -2.2735236, 0.45998785...
2982    [-2.2299173, 0.38330296, -2.260903, 0.84615386...
5117    [-1.2382377, 0.8447503, -2.495189, -0.28529474...
4480    [-1.9312211, 0.62989014, -1.582881, 0.6830024,...
Name: text_vectors, Length: 7920, dtype: object

In [45]:
from sklearn.preprocessing import MinMaxScaler
sca =MinMaxScaler()
x_train_scaled = sca.fit_transform(x_train_2d) 
x_test_scaled = sca.fit_transform(x_test_2d) 

In [46]:
x_test_scaled.shape


(1980, 300)

In [47]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train_scaled, y_train)

MultinomialNB()

In [48]:
clf.score(x_test_scaled,y_test)

0.9444444444444444

In [50]:
from sklearn.metrics import classification_report
y_pred = clf.predict(x_test_scaled)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95      1023
           1       0.92      0.96      0.94       957

    accuracy                           0.94      1980
   macro avg       0.94      0.95      0.94      1980
weighted avg       0.95      0.94      0.94      1980

